In [1]:
import tensorflow as tf 
from tensorflow.keras.layers import *
from transformers import BertTokenizer, TFBertForSequenceClassification
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# 构建训练集和测试集
EPOCH = 1000
BATCH_SIZE = 32
MAX_LENGTH = 128

BATH_PATH = "/home/wangchengkun/workspace/wechat_bert/"
origin_data = pd.read_csv(BATH_PATH + "data/social_support_new.csv", )
origin_data = np.array(origin_data)

# 处理x值，在这一步直接tokenizer
x_all = x_train = []
tokenizer = BertTokenizer.from_pretrained("bert-base-chinese")
for x in list(origin_data[:, 3]):
    x_all.append(tokenizer.encode(str(x), add_special_tokens=True, 
                 max_length=MAX_LENGTH, pad_to_max_length='right'))
# 处理y值
y_all = origin_data[:, 4]

# 划分x_train, x_test, y_train, y_test
x_train, x_test, y_train, y_test = train_test_split(x_all, y_all, random_state=42, 
                                                    test_size=0.05)

train_dataset = tf.data.Dataset.from_tensor_slices((x_train, list(y_train)))
train_dataset = train_dataset.batch(BATCH_SIZE)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, list(y_test)))
print("数据加载完成...")

数据加载完成...


In [2]:
from tensorflow.keras.callbacks import Callback
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
class Metrics(Callback):
    def on_train_begin(self, logs={}):
        self.val_f1s = []
        self.val_recalls = []
        self.val_precisions = []
    def on_epoch_end(self, epoch, logs={}):
        val_predict = (np.asarray(self.model.predict(self.validation_data[0]))).round()
        val_predict = np.argmax(val_predict, axis=1)
        val_targ = self.validation_data[1]
        _val_f1 = f1_score(val_targ, val_predict)
        _val_recall = recall_score(val_targ, val_predict)
        _val_precision = precision_score(val_targ, val_predict)
        self.val_f1s.append(_val_f1)
        self.val_recalls.append(_val_recall)
        self.val_precisions.append(_val_precision)
        print("\n— val_f1: %f — val_precision: %f — val_recall %f" %(_val_f1, _val_precision, _val_recall))
        return

In [ ]:
model = TFBertForSequenceClassification.from_pretrained("bert-base-chinese")
model.compile(loss="binary_crossentropy",
             optimizer = tf.keras.optimizers.Adam(1e-4),
             metrics=["accuracy"])
# 构建F1值计算工具
metrics = Metrics()
metrics.validation_data = (x_test, list(y_test))
print("模型构建完成...")
history = model.fit(train_dataset, epochs=EPOCH,
                    callbacks=[metrics],
                   validation_data=test_dataset,
                   validation_steps=BATCH_SIZE)

模型构建完成...
Epoch 1/1000
    402/Unknown - 151s 375ms/step - loss: 3.0994 - accuracy: 0.7989
— val_f1: 0.000000 — val_precision: 0.000000 — val_recall 0.000000
402/402 [==============================] - 157s 390ms/step - loss: 3.0994 - accuracy: 0.7989 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/1000
401/402 [============================>.] - ETA: 0s - loss: 3.1013 - accuracy: 0.7989
— val_f1: 0.000000 — val_precision: 0.000000 — val_recall 0.000000
402/402 [==============================] - 145s 362ms/step - loss: 3.1013 - accuracy: 0.7990 - val_loss: 2.4101 - val_accuracy: 0.8438
Epoch 3/1000
401/402 [============================>.] - ETA: 0s - loss: 3.1013 - accuracy: 0.7989
— val_f1: 0.000000 — val_precision: 0.000000 — val_recall 0.000000
402/402 [==============================] - 146s 363ms/step - loss: 3.1013 - accuracy: 0.7990 - val_loss: 2.4101 - val_accuracy: 0.8438
Epoch 4/1000
401/402 [============================>.] - ETA: 0s - loss: 3.1013 - accuracy: 0.7989
—

In [ ]:
print(history.history['val_loss'])